## Amazon Bedrock Prompt Flow Generation 

This notebook shows how we can take the Amazon Bedrock Knowledge Bases we created in `rag-router.ipynb` and put them in a structured flow using Amazon Bedrock Prompt Flows (https://aws.amazon.com/bedrock/prompt-flows/).

This will allow us to have a versioned flow where we can specify all of the sequential components, as well as any conditions we want to model. 

We create promtps we will store in 

We will start with a description of a RAG framework with additional modules (e.g., current date, web search, etc.) to generate a prompt flow as shown below.

** Generated Prompt Flow**

![alt text](prompt_flow_asset/example_pf_output.png "Automatically Generated Prompt Flow")

Let's start!

**Table of Contents:**

1. [Complete prerequisites](#Complete%20prerequisites)
    
    1. [Configure logging](#Configure%20logging)
        
        1. [System logs (Optional)](#Configure%20system%20logs%20(Optional))
        
        2. [Application logs](#Configure%20application%20logs)
    
    2. [Organize imports](#Organize%20imports)
    
    3. [Set AWS Region and boto3 config](#Set%20AWS%20Region%20and%20boto3%20config)
    
    4. [Create common objects](#Create%20common%20objects)
    
    5. [Get details of Knowledge Bases](#Get%20details%20of%20Knowledge%20Bases)

 2. [Create Prompts](#Load%20data%20to%20Knowledge%20Bases)
    
    1. [Step 0a: Create prompt for routing](#Load%20to%20KB%20Step0b)
    
    2. [Step 0b: Create prompt for AI Assistant](#Load%20to%20KB%20Steps0c%20to%200e)
 
 3. [Create Flow](#Process%20query)
 
     1. [Step 1: User query and non-KB query](#User%20query)
     
     2. [Steps 2a and 2b: Determine the KB id](#Determine%20the%20KB%20id)
     
     3. [Steps 3a through 5: Retrieve and generate](#Retrieve%20and%20generate)
 
 4. [Cleanup](#Cleanup)
 
 5. [Conclusion](#Conclusion)
 
 6. [Frequently Asked Questions (FAQs)](#FAQs)

####  b. Application logs <a id='Configure%20application%20logs'></a>

Application logs refers to the logs generated by running the various code cells in this notebook. To set this up, instantiate the [Python logging service](https://docs.python.org/3/library/logging.html) by running the following cell. You can configure the default log level and format as required.

By default, this notebook will only print the logs to the corresponding cell's output console.

In [ ]:
import logging
import os

# Set the logging level and format
log_level = logging.INFO
log_format = '%(asctime)s - %(levelname)s - %(message)s'
logging.basicConfig(level=log_level, format=log_format)

# Save these in the environment variables for use in the helper scripts
os.environ['LOG_LEVEL'] = str(log_level)
os.environ['LOG_FORMAT'] = log_format

###  B. Organize imports <a id ='Organize%20imports'> </a>


In [ ]:
import boto3
from datetime import datetime
import json
import sys
import os
import sagemaker

# Import the helper functions from the 'scripts' folder
sys.path.append(os.path.join(os.getcwd(), "scripts"))
#logging.info("Updated sys.path: {}".format(sys.path))
from helper_functions import *

role = sagemaker.get_execution_role()

bedrock_agent = boto3.client(service_name="bedrock-agent", region_name="us-west-2")

###  C. Get KB Details <a id ='Organize%20imports'> </a>

In [ ]:
kb_1_name = 'rag-router-kb-1'
kb_2_name = 'rag-router-kb-2'


kb_1_id, kb_1_ds_id, kb_1_s3_bucket_name, kb_1_aoss_collection_arn = get_kb_details(bedrock_agent, kb_1_name)
kb_2_id, kb_2_ds_id, kb_2_s3_bucket_name, kb_2_aoss_collection_arn = get_kb_details(bedrock_agent, kb_2_name)

###  2. Create Prompts <a id =Load%20data%20to%20Knowledge%20Bases> </a>

Let's create a prompt for our AI Assistant

In [ ]:
# Create the prompt
response = bedrock_agent.create_prompt(
    name=f"AIAssistantPrompt-{datetime.now().strftime('%Y%m%d-%H%M%S')}",
    description="AI Assistant prompt for Amazon Bedrock Knowledge Base",
    variants=[
        {
            "inferenceConfiguration": {
                "text": {
                    "maxTokens": 3000,
                    "temperature": 0,
                    "topP": 0.1,
                    # "topK": 250,
                }
            },
            "modelId": "anthropic.claude-3-haiku-20240307-v1:0",
            "name": "variant-001",
            "templateConfiguration": {
                "text": {
                    "inputVariables": [
                        {"name": "user_query"},
                        {"name": "kb_results"}
                    ],
                    "text": """You are an AI assistant who is well-versed in Amazon Bedrock Knowledge base designed to answer a user's question.
Please generate a code about Amazon Bedrock Knowledge Base only when a user asks for it.
Do NOT use your own knowledge as facts in answers. 
Please output a list in order of the most valuable data sources for a knowledge base when appropriate.
Don't restate the instructions.

User query: {{user_query}}

Knowledge base results: {{kb_results}}

Please provide a response based on the above information:"""
                }
            },
            "templateType": "TEXT"
        }
    ],
    defaultVariant="variant-001"
)

###  E. Create Prompts <a id =Load%20data%20to%20Knowledge%20Bases> </a>

Let's create a prompt for our AI Router

In [ ]:
router_response = bedrock_agent.create_prompt(
    name=f"AIRouterPrompt-{datetime.now().strftime('%Y%m%d-%H%M%S')}",
    description="AI Router prompt for Amazon Bedrock Knowledge Base",
    variants=[
        {
            "inferenceConfiguration": {
                "text": {
                    "maxTokens": 500,
                    "temperature": 0,
                    "topP": 0.1,
                    # "topK": 250,
                }
            },
            "modelId": "anthropic.claude-3-haiku-20240307-v1:0",
            "name": "variant-001",
            "templateConfiguration": {
                "text": {
                    "inputVariables": [
                        {"name": "QUERY"}
                    ],
                    "text": """Carefully take a look at the CATEGORY information specified in the <KBs> tag.
<KBs>
<CATEGORY1>code</CATEGORY2>
<CATEGORY1>docs</CATEGORY2>
</KBs>

Look at the query specified in the <QUERY> tag.

<QUERY>
{{QUERY}}
</QUERY>

Now, look at the output JSON format specified in the <OUTPUT> tag.

<OUTPUT>
{
  "category": "",
}
</OUTPUT>

Now, follow the instructions specified in the <INSTRUCTIONS> tag.
<INSTRUCTIONS>
- Identify the category for the query specified in the <QUERY> tag. It should be one of the values specified in the <CATEGORY> tags inside the <KBs> tag.
- Based on the identified category, create an output JSON message as specified in the <OUTPUT> tag with corresponding values for "category".
- If you do not know the answer, mention the "category" as "UNKNOWN".
- Your response should ONLY be a valid JSON as specified in the <OUTPUT> tag.
- Do not make up an answer.
- Do not include any preamble or postamble.
</INSTRUCTIONS>"""
                }
            },
            "templateType": "TEXT"
        }
    ],
    defaultVariant="variant-001"
)

In [ ]:
assistant_prompt_arn = assistant_response["arn"]
print(f"Assistant Prompt ARN: {assistant_prompt_arn}")
router_prompt_arn = router_response["arn"]
print(f"Router Prompt ARN: {router_prompt_arn}")

### Execution

After updating the generated code with your inputs on the required configurations, you can simple execute the generated code that is saved as a python file.

Please review the generated code before you execute it and see if everything is correct. 

If there was an error(s) in the code, the execution module will tell you what went wrong with the code.

In [ ]:

prompt_arn = response["arn"]
print(f"Prompt ARN: {prompt_arn}")

# Create the flow
flow_role = role #"arn:aws:iam::123456789012:role/BedrockFlowRole"  # Replace with your actual role ARN
kb_id = "" # Replace with your actual knowledge base ID

response = bedrock_agent.create_flow(
    name=f"BedrockKnowledgeBaseFlow-{datetime.now().strftime('%Y%m%d-%H%M%S')}",
    description="Flow for answering questions about Amazon Bedrock Knowledge Base",
    executionRoleArn=flow_role,
    definition={
        "nodes": [
            {
                "name": "UserInput",
                "type": "Input",
                "configuration": {
                    "input": {}
                },
                "outputs": [
                    {
                        "name": "document",
                        "type": "String"
                    }
                ],
            },
            {
                "name": "BedrockKnowledgeBase",
                "type": "KnowledgeBase",
                "configuration": {
                    "knowledgeBase": {
                        "knowledgeBaseId": kb_1_id  
                    }
                },
                "inputs": [
                    {
                        "name": "retrievalQuery",
                        "type": "String",
                        "expression": "$.data"
                    }
                ],
                "outputs": [
                    {
                        "name": "retrievalResults",
                        "type": "Array"
                    }
                ],
            },
            {
                "name": "AIAssistant",
                "type": "Prompt",
                "configuration": {
                    "prompt": {
                        "sourceConfiguration": {
                            "resource": {
                                "promptArn": assistant_prompt_arn
                            }
                        }
                    }
                },
                "inputs": [
                    {
                        "name": "user_query",
                        "type": "String",
                        "expression": "$.data"
                    },
                    {
                        "name": "kb_results",
                        "type": "Array",
                        "expression": "$.data"
                    }
                ],
                "outputs": [
                    {
                        "name": "modelCompletion",
                        "type": "String"
                    }
                ],
            },
            {
                "name": "FinalResponse",
                "type": "Output",
                "configuration": {
                    "output": {}
                },
                "inputs": [
                    {
                        "name": "document",
                        "type": "String",
                        "expression": "$.data"
                    }
                ],
            }
        ],
        "connections": [
            {
                "name": "Connection_1",
                "source": "UserInput",
                "target": "BedrockKnowledgeBase",
                "type": "Data",
                "configuration": {
                    "data": {
                        "sourceOutput": "document",
                        "targetInput": "retrievalQuery"
                    }
                }
            },
            {
                "name": "Connection_2",
                "source": "UserInput",
                "target": "AIAssistant",
                "type": "Data",
                "configuration": {
                    "data": {
                        "sourceOutput": "document",
                        "targetInput": "user_query"
                    }
                }
            },
            {
                "name": "Connection_3",
                "source": "BedrockKnowledgeBase",
                "target": "AIAssistant",
                "type": "Data",
                "configuration": {
                    "data": {
                        "sourceOutput": "retrievalResults",
                        "targetInput": "kb_results"
                    }
                }
            },
            {
                "name": "Connection_4",
                "source": "AIAssistant",
                "target": "FinalResponse",
                "type": "Data",
                "configuration": {
                    "data": {
                        "sourceOutput": "modelCompletion",
                        "targetInput": "document"
                    }
                }
            }
        ],
    }
)

flow_id = response["id"]
flow_arn = response["arn"]
flow_name = response["name"]
print(f"Flow ID: {flow_id}")
print(f"Flow ARN: {flow_arn}")
print(f"Flow Name: {flow_name}")

# Prepare the flow
response = bedrock_agent.prepare_flow(flowIdentifier=flow_id)
print(json.dumps(response, indent=2, default=str))

In [ ]:
version_response = bedrock_agent.create_flow_version(
    # clientToken='string',
    description='main-flow-version',
    flowIdentifier=flow_id
)
version_response

In [ ]:
alias_response = bedrock_agent.create_flow_alias(
    # clientToken='string',
    description='Main flow Alias',
    flowIdentifier=flow_id,
    name="main-flow-alias",
    routingConfiguration=[
        {
            'flowVersion': '1'
        },
    ],
    # tags={
    #     'string': 'string'
    # }
)
alias_response

In [ ]:
bedrock_agent_rt = boto3.client(service_name="bedrock-agent-runtime", region_name="us-west-2")

query = "tell me about KBs"

response = bedrock_agent_rt.invoke_flow(
    flowAliasIdentifier='0D8SVYHPBT',
    flowIdentifier=flow_id,
    inputs=[
        {
            'content': {
                'document': query
            },
            'nodeName': 'UserInput',
            'nodeOutputName': 'document'
        },
    ]
)
[response for response in iter(response['responseStream'])]

### Check the generated Prompt Flow 

After successfully executing the prompt flow deployment code, your prompt flow is deployed to your AWS account. 

Please go to your AWS console and go to Amazon Bedrock.

In the left panel, go to Prompt flows (preview) under Builder's tool.

Your prompt flow will be in the prompt flows list in the main panel. 

![alt text](prompt_flow_asset/example_pf_list.png "Prompt Flows List")


Click the generate prompt flow and Click Edit in prompt flow builder button in the top right corner.

![alt text](prompt_flow_asset/example_pf_panel.png "Prompt Flow Panel")

You will be able to see the draft of the generated prompt flow and test it with your questions about Amazon Bedrock Knowledge Base.

![alt text](prompt_flow_asset/example_pf_output.png "Automatically Generated Prompt Flow")

Thank you for following this example all the way to the end. Have fun building!